# RNN 모델의 작동 방식

## RNN

In [261]:
import torch
from torch import nn

# RNN 정의
rnn = nn.RNN(input_size=10, hidden_size=20, batch_first=True)

# 임의의 입력
x = torch.randn(1, 200, 10)  # 배치 크기: 1, 시퀀스 길이: 3, 입력 차원: 10
# t_1..t_200까지 순차적으로 10차원의 데이터가 들어가도록 만들겠다는 뜻.

# 모델 실행
output, hn = rnn(x)
# output은 모든 time 에서 나오는 모든 h들을 모아놓은 것
# hn은 200번에 해당하는 모든 데이터 포인트들을 다 지나간다음에 나오는 녀석

print(f"RNN output shape: {output.shape}")
print(f"RNN hn shape: {hn.shape}")


RNN output shape: torch.Size([1, 200, 20])
RNN hn shape: torch.Size([1, 1, 20])


In [262]:
output[0,-1,:]

tensor([ 0.2837,  0.5846, -0.7810, -0.1580, -0.5131,  0.5477,  0.3961, -0.2207,
        -0.6812,  0.4416, -0.2011, -0.7379,  0.0648,  0.1456,  0.3722,  0.2928,
        -0.5122,  0.2380, -0.6878,  0.2645], grad_fn=<SliceBackward0>)

In [263]:
hn[0,0,:]

tensor([ 0.2837,  0.5846, -0.7810, -0.1580, -0.5131,  0.5477,  0.3961, -0.2207,
        -0.6812,  0.4416, -0.2011, -0.7379,  0.0648,  0.1456,  0.3722,  0.2928,
        -0.5122,  0.2380, -0.6878,  0.2645], grad_fn=<SliceBackward0>)

## LSTM

In [264]:
# LSTM 정의
lstm = nn.LSTM(input_size=10, hidden_size=20, batch_first=True)

# 임의의 입력
x = torch.randn(1, 200, 10)  # 배치 크기: 1, 시퀀스 길이: 3, 입력 차원: 10
# t_1..t_200까지 순차적으로 10차원의 데이터가 들어가도록 만들겠다는 뜻.

# 모델 실행
output, (hn, cn) = lstm(x)
# 마찬가지로 output은 모든 time 에서 나오는 모든 h들을 모아놓은 것
# LSTM의 경우는 hidden state 에 더해서 channel state 도 있음에 유의

print(f"LSTM output shape: {output.shape}")
print(f"LSTM hn shape: {hn.shape}")
print(f"LSTM cn shape: {cn.shape}")


LSTM output shape: torch.Size([1, 200, 20])
LSTM hn shape: torch.Size([1, 1, 20])
LSTM cn shape: torch.Size([1, 1, 20])


In [265]:
output[0,-1,:]

tensor([-0.0045, -0.0431, -0.0748, -0.0456, -0.1018,  0.1048, -0.0256,  0.1841,
         0.0415,  0.2195,  0.0911, -0.2329,  0.0323,  0.1803,  0.0380,  0.0689,
         0.1565, -0.0906,  0.2033, -0.1187], grad_fn=<SliceBackward0>)

In [266]:
hn[0,0,:]

tensor([-0.0045, -0.0431, -0.0748, -0.0456, -0.1018,  0.1048, -0.0256,  0.1841,
         0.0415,  0.2195,  0.0911, -0.2329,  0.0323,  0.1803,  0.0380,  0.0689,
         0.1565, -0.0906,  0.2033, -0.1187], grad_fn=<SliceBackward0>)

In [267]:
cn[0,0,:]

tensor([-0.0126, -0.0754, -0.1890, -0.1230, -0.2577,  0.1880, -0.0419,  0.3580,
         0.1312,  0.5014,  0.2118, -0.5459,  0.0674,  0.4726,  0.0581,  0.1425,
         0.2614, -0.1892,  0.5311, -0.2322], grad_fn=<SliceBackward0>)

## 일반적인 모델의 형태

In [268]:
class Model(nn.Module):
    def __init__(self, input_size, hid_size, output_size):
        super(Model, self).__init__()
        self.LSTM = nn.LSTM(input_size = input_size, hidden_size = hid_size, num_layers=2,
                            batch_first=True)   
        self.fc1 = nn.Linear(self.hid_size, self.hid_size)
        self.fc2 = nn.Linear(self.hid_size, output_size)
    
    def forward(self, x):
        """
        input : [bs, maxlen]
        output: [bs, 2] 
        """
        x = self.Embedding(x)  # [bs, ml, emb_size]
        x, (h,c) = self.LSTM(x)  # [bs, ml, hid_size]
        x = F.relu(self.fc1(x[:,-1,:]))   # [bs, ml, hid_size]
        # x = F.relu(self.fc1(h))   # [bs, ml, hid_size]
        out = self.fc2(x)  
        return out  # [bs, output_size]

# RNN모델을 통한 주가 예측 (regression)

In [269]:
!pip install yfinance scikit-learn

In [ ]:
import torch
import torch.nn as nn
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 데이터 불러오기
data = yf.download('AAPL', '2000-01-01', '2023-07-01')
data = data['Close'].values

# 데이터 정규화
scaler = MinMaxScaler()
data = scaler.fit_transform(data.reshape(-1, 1)).reshape(-1)

# 설정값 정의
seq_length = 2000
pred_length = 200
hidden_dim = 50
n_features = 1

# 학습 데이터 생성
def create_sequences(data, seq_length, pred_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-pred_length):
        x = data[i:(i+seq_length)]
        y = data[(i+seq_length):(i+seq_length+pred_length)]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

x, y = create_sequences(data, seq_length, pred_length)
x = torch.tensor(x).float().view(-1, seq_length, n_features)
y = torch.tensor(y).float()

train_x = x[:len(x)//2]
test_x = x[len(x)//2:]
train_y = y[:len(y)//2]
test_y = y[len(y)//2:]

# 모델 정의
class StockPredictor(nn.Module):
    def __init__(self, n_features, hidden_dim, seq_length, pred_length):
        super(StockPredictor, self).__init__()
        self.n_features = n_features
        self.hidden_dim = hidden_dim
        self.seq_len = seq_length
        self.pred_len = pred_length
        self.lstm = nn.LSTM(input_size=n_features, hidden_size=hidden_dim)
        self.linear = nn.Linear(in_features=hidden_dim, out_features=pred_length)

    def forward(self, sequences):
        lstm_out, _ = self.lstm(sequences.view(len(sequences), self.seq_len, -1))
        last_time_step = lstm_out.view(self.seq_len, len(sequences), self.hidden_dim)[-1]
        y_pred = self.linear(last_time_step)
        return y_pred

model = StockPredictor(n_features, hidden_dim, seq_length, pred_length)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 학습
model.train()
for epoch in range(50):  # 50번의 epoch 동안 학습
    model.zero_grad()
    y_pred = model(train_x)
    loss = criterion(y_pred.float(), train_y)
    print("Epoch ", epoch, "MSE: ", loss.item())
    loss.backward()
    optimizer.step()

# 평가
model.eval()
test_preds = model(test_x)


[*********************100%***********************]  1 of 1 completed
Epoch  0 MSE:  0.013738235458731651
Epoch  1 MSE:  0.012976775877177715
Epoch  2 MSE:  0.012270951643586159
Epoch  3 MSE:  0.011611342430114746
Epoch  4 MSE:  0.010990006849169731
Epoch  5 MSE:  0.010399633087217808
Epoch  6 MSE:  0.00983443483710289
Epoch  7 MSE:  0.00929019320756197
Epoch  8 MSE:  0.00876376498490572
Epoch  9 MSE:  0.008252782747149467
Epoch  10 MSE:  0.007755504455417395
Epoch  11 MSE:  0.007270782254636288
Epoch  12 MSE:  0.006798103451728821
Epoch  13 MSE:  0.006337698549032211
Epoch  14 MSE:  0.005890676286071539
Epoch  15 MSE:  0.00545924948528409
Epoch  16 MSE:  0.005047030746936798
Epoch  17 MSE:  0.004659315105527639
Epoch  18 MSE:  0.00430311867967248
Epoch  19 MSE:  0.0039863986894488335
Epoch  20 MSE:  0.003715324215590954
Epoch  21 MSE:  0.0034894058480858803
Epoch  22 MSE:  0.003297035349532962
Epoch  23 MSE:  0.003117527114227414


In [ ]:
%matplotlib inline

plt.figure(figsize=(16,6))

range_future = len(test_y)

plt.plot(np.arange(200), test_y[0,:].numpy(), label='True Future')
plt.plot(np.arange(200), test_preds[0,:].detach().numpy(), label='Predicted Future')

plt.title('True Future vs Predicted Future')
plt.xlabel('Time Steps')
plt.ylabel('Normalized Stock Price')
plt.legend()


# RNN을 이용한 문서의 분류 (classification)

In [ ]:
!pip install tensorflow

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torch.utils.data import * 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.datasets import imdb 

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_WORDS)
x_train = pad_sequences(x_train, maxlen=MAX_LEN, padding="post", truncating="post")
x_test = pad_sequences(x_test, maxlen=MAX_LEN, padding="post", truncating="post")
print(x_train.shape, x_test.shape)

In [ ]:
train_data = TensorDataset(torch.LongTensor(x_train), torch.LongTensor(y_train))
test_data = TensorDataset(torch.LongTensor(x_test), torch.LongTensor(y_test))

train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

test_sampler = SequentialSampler(test_data)
test_loader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
class Model(nn.Module):
    def __init__(self, max_words, emb_size, hid_size):
        super(Model, self).__init__()
        self.max_words = max_words
        self.emb_size = emb_size
        self.hid_size = hid_size
        self.Embedding = nn.Embedding(self.max_words, self.emb_size)
        self.LSTM = nn.LSTM(self.emb_size, self.hid_size, num_layers=2,
                            batch_first=True)   
        self.fc1 = nn.Linear(self.hid_size, self.hid_size)
        self.fc2 = nn.Linear(self.hid_size, 2)
    
    def forward(self, x):
        """
        input : [bs, maxlen]
        output: [bs, 2] 
        """
        x = self.Embedding(x)  # [bs, ml, emb_size]
        x, (h,c) = self.LSTM(x)  # [bs, ml, hid_size]
        x = F.relu(self.fc1(x[:,-1,:]))   # [bs, ml, hid_size]
        out = self.fc2(x)  
        return out  # [bs, 2]

In [ ]:
def train(model, device, train_loader, optimizer, epoch):   
    model.train()
    criterion = nn.CrossEntropyLoss()
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(DEVICE), y.to(DEVICE)
                
        optimizer.zero_grad()
        y_ = model(x)
        loss = criterion(y_, y)  # loss
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(x), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(model, device, test_loader): 
    model.eval()
    criterion = nn.CrossEntropyLoss(reduction='sum')  
    test_loss = 0.0 
    acc = 0 
    for batch_idx, (x, y) in enumerate(test_loader):
        x, y = x.to(DEVICE), y.to(DEVICE)
        with torch.no_grad():
            y_ = model(x)
        test_loss += criterion(y_, y)
        pred = y_.max(-1, keepdim=True)[1] 
        acc += pred.eq(y.view_as(pred)).sum().item()    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, acc, len(test_loader.dataset),
        100. * acc / len(test_loader.dataset)))
    return acc / len(test_loader.dataset)

In [ ]:
MAX_WORDS = 10000  # imdb’s vocab_size 
MAX_LEN = 200      # max length
BATCH_SIZE = 256
EMB_SIZE = 128   # embedding size
HID_SIZE = 128   # lstm hidden size


model = Model(MAX_WORDS, EMB_SIZE, HID_SIZE).to(DEVICE)
print(model)
optimizer = optim.Adam(model.parameters())

best_acc = 0.0 
PATH = 'imdb model/model.pth'  #

for epoch in range(10):
    train(model, DEVICE, train_loader, optimizer, epoch)
    acc = test(model, DEVICE, test_loader)
    if best_acc < acc: 
        best_acc = acc 
        torch.save(model.state_dict(), PATH)
    print("acc is: {:.4f}, best acc is {:.4f}\n".format(acc, best_acc)) 